# <center>PROJECT SANDBOX</center>

The aim of this notebook is to provide a simple sandbox to test different NN architectures for the project. , 3 functions are imported from the `scripts` folder : 

- `prepare_dataset()` : input a torch.device and output a preprocessed torch Dataset and a tokenizer dictionnary
- `string2code()` : input  a string sentence and a tokenizer to output a torch.Longtensor (sentence tokenized)
- `code2string()` : input  a torch.Longtensor (sentence tokenized) and a tokenizer to output a string sentence

## Importing packages

In [1]:
from scripts.data_builders.prepare_dataset import prepare_dataset,string2code,code2string

import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import ipdb
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ",device)

device =  cpu


## Preprocessing data

In [2]:
train_data, dict_words = prepare_dataset(device) #check with shift+tab to look at the data structure
batch_size = 10

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           shuffle=True,collate_fn=train_data.collate)

Loading ...
- Shakespeare dataset length :  20316
- Corrupted samples (ignored) :  763


## Designing NN model

## Running model

In [3]:
word_dict={v:k for k,v in dict_words.items()}

In [4]:
for x,y , ctx_x,ctx_y , len_x,len_y , len_ctx_x,len_ctx_y in train_loader:
    print(code2string(x[0],word_dict))
    print(code2string(y[0],word_dict))
    print(code2string(ctx_x[0],word_dict))
    break
#     print(y)
#     ipdb.set_trace()

KeyError: -1